<center>
<img src="https://habrastorage.org/webt/ia/m9/zk/iam9zkyzqebnf_okxipihkgjwnw.jpeg">
     
## <center>  [mlcourse.ai](https://mlcourse.ai) – Open Machine Learning Course 

#### <center> Author: [Yury Kashnitsky](https://yorko.github.io) (@yorko) 

# <center>Assignment #2. Fall 2019. Solution
## <center> Part 2. Gradient boosting

Beating benchmarks in [this competition](https://www.kaggle.com/c/flight-delays-fall-2018/overview).

In [1]:
import numpy as np
import pandas as pd
from pathlib import Path
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from catboost import CatBoostClassifier

**Read the data**

In [2]:
PATH_TO_DATA = Path(r'C:\Users\One\Git\ml-ds')

In [11]:
train_df = pd.read_csv(r'C:\Users\One\Git\ml-ds\as2\flight_delays_train.csv.zip', compression = 'zip')
test_df = pd.read_csv(r'C:\Users\One\Git\ml-ds\as2\flight_delays_test.csv.zip', compression = 'zip')

In [12]:
train_df.shape

(100000, 9)

In [13]:
train_df.head()

,Month,DayofMonth,DayOfWeek,DepTime,UniqueCarrier,Origin,Dest,Distance,dep_delayed_15min
0,c-8,c-21,c-7,1934,AA,ATL,DFW,732,N
1,c-4,c-20,c-3,1548,US,PIT,MCO,834,N
2,c-9,c-2,c-5,1422,XE,RDU,CLE,416,N
3,c-11,c-25,c-6,1015,OO,DEN,MEM,872,N
4,c-10,c-7,c-6,1828,WN,MDW,OMA,423,Y


In [14]:
test_df.head()

,Month,DayofMonth,DayOfWeek,DepTime,UniqueCarrier,Origin,Dest,Distance
0,c-7,c-25,c-3,615,YV,MRY,PHX,598
1,c-4,c-17,c-2,739,WN,LAS,HOU,1235
2,c-12,c-2,c-7,651,MQ,GSP,ORD,577
3,c-3,c-25,c-7,1614,WN,BWI,MHT,377
4,c-6,c-6,c-3,1505,UA,ORD,STL,258


**Create features**

Mostly from [this Kernel](https://www.kaggle.com/rohitgr/lgbm-bayesianoptimization-eda)

In [15]:
# flight
train_df['flight'] = train_df['Origin'] + '-->' + train_df['Dest']
test_df['flight'] = test_df['Origin'] + '-->' + test_df['Dest']

# Hour and minute
train_df['hour'] = train_df['DepTime'] // 100
train_df.loc[train_df['hour'] == 24, 'hour'] = 0
train_df.loc[train_df['hour'] == 25, 'hour'] = 1
train_df['minute'] = train_df['DepTime'] % 100

test_df['hour'] = test_df['DepTime'] // 100
test_df.loc[test_df['hour'] == 24, 'hour'] = 0
test_df.loc[test_df['hour'] == 25, 'hour'] = 1
test_df['minute'] = test_df['DepTime'] % 100

# Season
train_df['summer'] = (train_df['Month'].isin([6, 7, 8])).astype(np.int32)
train_df['autumn'] = (train_df['Month'].isin([9, 10, 11])).astype(np.int32)
train_df['winter'] = (train_df['Month'].isin([12, 1, 2])).astype(np.int32)
train_df['spring'] = (train_df['Month'].isin([3, 4, 5])).astype(np.int32)

test_df['summer'] = (test_df['Month'].isin([6, 7, 8])).astype(np.int32)
test_df['autumn'] = (test_df['Month'].isin([9, 10, 11])).astype(np.int32)
test_df['winter'] = (test_df['Month'].isin([12, 1, 2])).astype(np.int32)
test_df['spring'] = (test_df['Month'].isin([3, 4, 5])).astype(np.int32)

# Daytime
train_df['daytime'] = pd.cut(train_df['hour'], bins=[0, 6, 12, 18, 23], include_lowest=True)
test_df['daytime'] = pd.cut(test_df['hour'], bins=[0, 6, 12, 18, 23], include_lowest=True)

# Extract the labels
train_y = train_df.pop('dep_delayed_15min')
train_y = train_y.map({'N': 0, 'Y': 1})

# Concatenate for preprocessing
train_split = train_df.shape[0]
full_df = pd.concat((train_df, test_df))
full_df['Duration'] = round(full_df['Distance']/7).astype(np.int32)

full_df['LogDistance'] = np.log(full_df['Distance'])

#full_df['Arrival'] = ((full_df['DepTime'] + full_df['Duration']) // 100)
#full_df.loc[full_df['Arrival'] >=24, 'Arrival'] = 1



full_df['Avaiting2'] = (full_df['Duration'] / np.log(full_df['Duration'])).astype(np.int32)



#количество рейсов у аэродрома посадки
Dest_int = train_df.groupby('Dest').count()
df = Dest_int.sort_values(by = ['Month'], ascending = [False])

df['Dest_intense'] = np.where(df['Month'] >=5000, 'a', 
                        (np.where(df['Month'] >=4000, 'b', 
                            (np.where((df['Month'] >= 3000), 'c', 
                                     (np.where((df['Month'] >= 2000), 'd', 
                                               (np.where((df['Month'] >= 1500), 'e', 
                                                        (np.where((df['Month'] >= 1000), 'f', 
                                                                 (np.where((df['Month'] >= 500), 'g', 'h')))))))))))))

a = list(df[df['Dest_intense'] == 'a'].index)
b = list(df[df['Dest_intense'] == 'b'].index)
c = list(df[df['Dest_intense'] == 'c'].index)
d = list(df[df['Dest_intense'] == 'd'].index)
e = list(df[df['Dest_intense'] == 'e'].index)
f = list(df[df['Dest_intense'] == 'f'].index)
g = list(df[df['Dest_intense'] == 'g'].index)
h = list(df[df['Dest_intense'] == 'h'].index)

full_df['Dest_intense'] = np.where(full_df['Dest'].isin(a), 'a',
                                   (np.where(full_df['Dest'].isin(b), 'b',
                                             (np.where(full_df['Dest'].isin(c), 'c',
                                                      (np.where(full_df['Dest'].isin(d), 'd',
                                                               (np.where(full_df['Dest'].isin(e), 'e',
                                                                        (np.where(full_df['Dest'].isin(f), 'f',
                                                                                 (np.where(full_df['Dest'].isin(g), 'g', 'h')))))))))))))



In [16]:
#количество рейсоы у аэродрома вылета
Origin_int = train_df.groupby('Origin').count()
df2 = Origin_int.sort_values(by = ['Month'], ascending = [False])

df2['Origin_intense'] = np.where(df2['Month'] >=5000, 'a2', 
                        (np.where(df2['Month'] >=4000, 'b2', 
                            (np.where((df2['Month'] >= 3000), 'c2', 
                                     (np.where((df2['Month'] >= 2000), 'd2', 
                                               (np.where((df2['Month'] >= 1500), 'e2', 
                                                        (np.where((df2['Month'] >= 1000), 'f2', 
                                                                 (np.where((df2['Month'] >= 500), 'g2', 'h2')))))))))))))
a2 = list(df2[df2['Origin_intense'] == 'a2'].index)
b2 = list(df2[df2['Origin_intense'] == 'b2'].index)
c2 = list(df2[df2['Origin_intense'] == 'c2'].index)
d2 = list(df2[df2['Origin_intense'] == 'd2'].index)
e2 = list(df2[df2['Origin_intense'] == 'e2'].index)
f2 = list(df2[df2['Origin_intense'] == 'f2'].index)
g2 = list(df2[df2['Origin_intense'] == 'g2'].index)
h2 = list(df2[df2['Origin_intense'] == 'h2'].index)
full_df['Origin_intense'] = np.where(full_df['Origin'].isin(a2), 'a2',
                                   (np.where(full_df['Origin'].isin(b2), 'b2',
                                             (np.where(full_df['Origin'].isin(c2), 'c2',
                                                      (np.where(full_df['Origin'].isin(d2), 'd2',
                                                               (np.where(full_df['Origin'].isin(e2), 'e2',
                                                                        (np.where(full_df['Origin'].isin(f2), 'f2',
                                                                                 (np.where(full_df['Origin'].isin(g2), 'g2', 'h2')))))))))))))
full_df['Intense'] = full_df['Origin_intense'] + '-->' + full_df['Dest_intense']

# String to numerical
for col in ['Month', 'DayofMonth', 'DayOfWeek']:
    full_df[col] = full_df[col].apply(lambda x: x.split('-')[1]).astype(np.int32) - 1

# Label Encoding
for col in ['Origin', 'Dest', 'UniqueCarrier', 'daytime', 'flight', 'Duration', 'LogDistance']:
    full_df[col] = pd.factorize(full_df[col])[0]

# Categorical columns
cat_cols = ['Month', 'DayofMonth', 'DayOfWeek', 'Origin', 'Dest', 'UniqueCarrier', 
            'hour', 'summer', 'autumn', 'winter', 'spring', 'daytime', 'flight', 
            'Origin_intense', 'Dest_intense', 'Intense']

# Converting categorical columns to type 'category' as required by LGBM
for c in cat_cols:
    full_df[c] = full_df[c].astype('category')

# Split into train and test
train_df, test_df = full_df.iloc[:train_split], full_df.iloc[train_split:]
train_df.shape, train_y.shape, test_df.shape

((100000, 22), (100000,), (100000, 22))

**Remember the indexes of categorical features**

In [17]:
train_df.dtypes

Month             category
DayofMonth        category
DayOfWeek         category
DepTime              int64
UniqueCarrier     category
Origin            category
Dest              category
Distance             int64
flight            category
hour              category
minute               int64
summer            category
autumn            category
winter            category
spring            category
daytime           category
Duration             int64
LogDistance          int64
Avaiting2            int32
Dest_intense      category
Origin_intense    category
Intense           category
dtype: object

In [18]:
categ_feat_idx = np.where(train_df.dtypes == 'category')[0]
categ_feat_idx

array([ 0,  1,  2,  4,  5,  6,  8,  9, 11, 12, 13, 14, 15, 19, 20, 21],
      dtype=int64)

In [19]:
from sklearn.model_selection import StratifiedKFold, cross_val_score

In [20]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=17)

In [21]:
former_feats = ['Month', 'DayofMonth', 'DayOfWeek', 'DepTime', 'UniqueCarrier', 
                'Origin', 'Dest', 'Distance', 'flight', 'hour', 'minute']
new_feats = ['summer', 'autumn', 'winter', 'spring', 'LogDistance',
             'Origin_intense', 'Dest_intense', 'Intense', 'Duration', 'Arrival']

In [22]:
ctb = CatBoostClassifier(random_seed=17, silent=True)

In [23]:
%%time
features_to_use = former_feats[:]
initial_cv_scores = cross_val_score(ctb, train_df[features_to_use].values, train_y, cv=skf, scoring='roc_auc', 
                fit_params={'cat_features': np.where(train_df[features_to_use].dtypes == 'category')[0]})
initial_cv_scores, initial_cv_scores.mean()

Wall time: 9min 34s


In [35]:
ctb.fit(train_df[new_feats[:]].values, train_y)

KeyError: "['Arrival'] not in index"

In [31]:
ctb.feature_importances_

array([ 8.51620513,  7.02838228,  4.91145342, 18.0507282 ,  9.10663975,
        9.78346729,  8.25652576,  8.90021571,  5.30289402, 13.5953692 ,
        6.54811923])

**Now we are performing cross-validation addigng features one by one**

In [24]:
%%time
cv_scores = []

features_to_use = former_feats[:]

for new_feat in tqdm(new_feats):
    print(f'adding feature {new_feat}')
    
    features_to_use += [new_feat]
    print(f'performing CV with features: {features_to_use}')

    current_cv_scores = cross_val_score(ctb, train_df[features_to_use].values, train_y, cv=skf, scoring='roc_auc', 
                fit_params={'cat_features': np.where(train_df[features_to_use].dtypes == 'category')[0]})
    print('5-fold CV is : {} +/- {}'.format(current_cv_scores.mean().round(4), 
                                            current_cv_scores.std().round(4)))
    cv_scores.append(current_cv_scores)
    

  0%|                                                   | 0/10 [00:00<?, ?it/s]

adding feature summer
performing CV with features: ['Month', 'DayofMonth', 'DayOfWeek', 'DepTime', 'UniqueCarrier', 'Origin', 'Dest', 'Distance', 'flight', 'hour', 'minute', 'summer']
5-fold CV is : 0.8156 +/- 0.0049


 10%|████                                    | 1/10 [09:42<1:27:21, 582.44s/it]

adding feature autumn
performing CV with features: ['Month', 'DayofMonth', 'DayOfWeek', 'DepTime', 'UniqueCarrier', 'Origin', 'Dest', 'Distance', 'flight', 'hour', 'minute', 'summer', 'autumn']
5-fold CV is : 0.8156 +/- 0.0049


 20%|████████                                | 2/10 [19:23<1:17:35, 581.98s/it]

...

 40%|████████████████▊                         | 4/10 [38:14<57:12, 572.00s/it]

adding feature LogDistance
performing CV with features: ['Month', 'DayofMonth', 'DayOfWeek', 'DepTime', 'UniqueCarrier', 'Origin', 'Dest', 'Distance', 'flight', 'hour', 'minute', 'summer', 'autumn', 'winter', 'spring', 'LogDistance']
5-fold CV is : 0.8143 +/- 0.0052


 50%|█████████████████████                     | 5/10 [47:37<47:25, 569.06s/it]

adding feature Origin_intense
performing CV with features: ['Month', 'DayofMonth', 'DayOfWeek', 'DepTime', 'UniqueCarrier', 'Origin', 'Dest', 'Distance', 'flight', 'hour', 'minute', 'summer', 'autumn', 'winter', 'spring', 'LogDistance', 'Origin_intense']


KeyboardInterrupt: 

In [25]:
a = ctb.feature_importances_
a

array(None, dtype=object)

In [26]:
from matplotlib import pyplot as plt
%config InlineBackend.figure_format='retina'

In [27]:
mean_cv_scores = np.array(cv_scores).mean(axis=1)
mean_cv_scores_minus_std = np.array(cv_scores).mean(axis=1) - np.array(cv_scores).std(axis=1)
mean_cv_scores_plus_std = np.array(cv_scores).mean(axis=1) + np.array(cv_scores).std(axis=1)

NameError: name 'cv_scores' is not defined

In [ ]:
plt.figure(figsize=(12, 8))
plt.plot(new_feats, mean_cv_scores, label='mean');
plt.plot(new_feats, mean_cv_scores_minus_std, c='green', linestyle='dashed');
plt.plot(new_feats, mean_cv_scores_plus_std, c='green', linestyle='dashed');
plt.fill_between(new_feats, mean_cv_scores_minus_std, mean_cv_scores_plus_std,
                 color='green', alpha=.1, label='std')
plt.xlabel('new feature')
plt.ylabel('CV score (mean +/- std)')
plt.grid(True)
plt.legend();

**Now test predictions, again adding features one by one to former 9 ones**

In [ ]:
sample_sub = pd.read_csv(r'C:\Users\One\Git\ml-ds\as2\sample_submission.csv.zip', compression = 'zip', index_col='id')

In [ ]:
%%time
features_to_use = former_feats[:]

for i, new_feat in tqdm(enumerate(new_feats)):
    print(f'adding feature {new_feat}')
    
    features_to_use += [new_feat]
    
    print(f'Training with features: {features_to_use}')
    
    cat_feat_idx = np.where(train_df[features_to_use].dtypes == 'category')[0]
    
    ctb.fit(train_df[features_to_use].values, train_y,cat_features=cat_feat_idx)
    ctb_test_pred = ctb.predict_proba(test_df[features_to_use].values)[:, 1]
    
    sample_sub['dep_delayed_15min'] = ctb_test_pred
    sub_file_name = f'ctb_pred_old_feats_and_{i+1}_new_ones.csv'
    sample_sub.to_csv('submit_36.csv')

In [34]:
ctb.predict_proba(test_df[features_to_use].values)[:, 1]
sample_sub['dep_delayed_15min'] = ctb_test_pred
sample_sub.to_csv('submit_36.csv')

NameError: name 'features_to_use' is not defined

**Analyzing CV & LB scores**

|          | Old feature set | +hour   | +minute | +summer | +autumn | +winter | +spring | +daytime |
|----------|-----------------|---------|---------|---------|---------|---------|---------|----------|
| Mean CV score | 0.765         | 0.789  | 0.812  | 0.821  | 0.82  | 0.822   | 0.821  | 0.821   |
| LB score | 0.73496         | 0.74505 | 0.76007 | 0.76136 | 0.76200 | 0.76014 | 0.76093 | 0.76082  |





CV scores have standard deviation from 0.001 to 0.003 (see results above), i.e mean CV scores can be reported with precision up to 3 decimals. Also, results might differ a bit from run to run. 

In [38]:
result_df = pd.DataFrame({'mean_cv_score': [0.765, 0.789, 0.82, 0.821, 0.82, 0.822, 0.821, 0.821],
                          'lb_score':      [0.73496, 0.74505, 0.76007, 0.76136, 0.76200, 0.76014, 0.76093, 0.76082 ]})

In [39]:
result_df.corr()

,mean_cv_score,lb_score
mean_cv_score,1.000000,0.996372
lb_score,0.996372,1.000000


We see an almost perfect correlation between cross-validation and leaderboard scores. That means that cross-validation is set-up fine.